In [1]:
import findspark
findspark.init('/opt/spark')
from pyspark.sql import SparkSession

In [2]:
from pyspark.sql.functions import col, broadcast

In [3]:
spark = SparkSession.builder\
    .enableHiveSupport()\
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/01/09 08:52:01 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [13]:
! hdfs dfs -mkdir dz5

2025-01-04 14:08:58,776 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [14]:
! hdfs dfs -put crime.csv dz5/crime.csv

2025-01-04 14:09:02,615 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [20]:
! hdfs dfs -put offense_codes.csv dz5/offense_codes.csv

2025-01-04 14:11:19,699 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
df_crime = spark.read.option("header", True).csv("dz5/crime.csv")

In [5]:
df_crime.show(3)

+---------------+------------+------------------+-------------------+--------+--------------+--------+-------------------+----+-----+-----------+----+----------+-----------+-----------+------------+--------------------+
|INCIDENT_NUMBER|OFFENSE_CODE|OFFENSE_CODE_GROUP|OFFENSE_DESCRIPTION|DISTRICT|REPORTING_AREA|SHOOTING|   OCCURRED_ON_DATE|YEAR|MONTH|DAY_OF_WEEK|HOUR|  UCR_PART|     STREET|        Lat|        Long|            Location|
+---------------+------------+------------------+-------------------+--------+--------------+--------+-------------------+----+-----+-----------+----+----------+-----------+-----------+------------+--------------------+
|     I182070945|       00619|           Larceny| LARCENY ALL OTHERS|     D14|           808|    null|2018-09-02 13:00:00|2018|    9|     Sunday|  13|  Part One| LINCOLN ST|42.35779134|-71.13937053|(42.35779134, -71...|
|     I182070943|       01402|         Vandalism|          VANDALISM|     C11|           347|    null|2018-08-21 00:00:0

In [6]:
df_crime.count()

319073

In [7]:
df_offense = spark.read.option("header", True).csv("dz5/offense_codes.csv")

In [8]:
df_offense.show(3)

+----+--------------------+
|CODE|                NAME|
+----+--------------------+
| 612|LARCENY PURSE SNA...|
| 613| LARCENY SHOPLIFTING|
| 615|LARCENY THEFT OF ...|
+----+--------------------+
only showing top 3 rows



In [9]:
df_offense.count()

576

In [36]:
from pyspark.sql.functions import col, broadcast

In [14]:
broadcast_test = (df_crime
    .join(broadcast(df_offense), df_offense["CODE"] == df_crime["OFFENSE_CODE"])
    #.filter(col("NAME").startswith("ROBBERY"))
    .groupBy("DISTRICT")
    .count()
    #.orderBy(col("count").desc())
)

broadcast_test.show(1)

+--------+-----+
|DISTRICT|count|
+--------+-----+
|      A1|   44|
+--------+-----+
only showing top 1 row



+------------+-----+
|OFFENSE_CODE|count|
+------------+-----+
|        3115|   16|
|        1601|    4|
|        2407|    2|
|        1605|    2|
|        1501|    2|
+------------+-----+
only showing top 5 rows



In [16]:
from pyspark.sql import functions

In [22]:
#crimes_total - общее количество преступлений в этом районе
crimes_total = df_crime.groupBy("DISTRICT").agg(functions.count("INCIDENT_NUMBER").alias("crime_count"))
crimes_total.show(7)

+--------+-----------+
|DISTRICT|crime_count|
+--------+-----------+
|      C6|      23460|
|    null|       1765|
|      B2|      49945|
|     C11|      42530|
|     E13|      17536|
|      B3|      35442|
|      E5|      13239|
+--------+-----------+
only showing top 7 rows



In [31]:
#crimes_monthly - медиана числа преступлений в месяц в этом районе
crimes_monthly_med = df_crime.groupBy("DISTRICT", "YEAR", "MONTH").agg(functions.count("INCIDENT_NUMBER").alias("crime_per_month"))
crimes_monthly = crimes_monthly_med.groupBy("DISTRICT", "YEAR", "MONTH").agg(functions.percentile_approx("crime_per_month", 0.5).alias("median_crime_per_month"))
crimes_monthly.filter(col("DISTRICT").isNotNull()).show(6)

+--------+----+-----+----------------------+
|DISTRICT|YEAR|MONTH|median_crime_per_month|
+--------+----+-----+----------------------+
|      A1|2015|   10|                  1031|
|      A1|2015|   11|                   838|
|      A1|2015|   12|                   832|
|      A1|2015|    6|                   445|
|      A1|2015|    7|                   889|
|      A1|2015|    8|                   966|
+--------+----+-----+----------------------+
only showing top 6 rows



In [34]:
#lat - широта координаты района, расчитанная как среднее по всем широтам инцидентов
lat = df_crime.groupBy("DISTRICT").agg(functions.avg("Lat").alias("avg_lat"))
lat.show(5)

+--------+------------------+
|DISTRICT|           avg_lat|
+--------+------------------+
|      C6| 42.21212258445509|
|    null| 25.23950519369344|
|      B2| 42.31600367732632|
|     C11| 42.29263740899965|
|     E13|42.309803655709324|
+--------+------------------+
only showing top 5 rows



In [35]:
#lng - долгота координаты района, расчитанная как среднее по всем долготам инцидентов
lng = df_crime.groupBy("DISTRICT").agg(functions.avg("Long").alias("avg_long"))
lng.show(5)

+--------+------------------+
|DISTRICT|          avg_long|
+--------+------------------+
|      C6|-70.85561011772268|
|    null|-43.44877438704257|
|      B2| -71.0756993065438|
|     C11|-71.05125995734306|
|     E13|-71.09800478878299|
+--------+------------------+
only showing top 5 rows

